In [1]:
from os import environ
environ['HF_HOME'] = '/Volumes/DATA/hf_home'
environ['HTTP_PROXY'] = 'http://localhost:7890'
environ['HTTPS_PROXY'] = 'http://localhost:7890'
environ['OPENAI_API_BASE'] = 'http://10.1.3.28:8990/v1'
environ['OPENAI_API_KEY'] = 'sk-'

from retrieval_qa_benchmark.datasets import MMLU
from retrieval_qa_benchmark.datasets import HotpotQA
from retrieval_qa_benchmark.datasets import WikiHop


/Users/mpsk/Library/Caches/pypoetry/virtualenvs/retrieval-qa-benchmark-OUwPJWBa-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from retrieval_qa_benchmark.utils.transforms import MultipleChoiceTransform
# dataset = HotpotQA.build(subset='fullwiki')
# dataset = WikiHop.build(subset='original')
dataset = MMLU.build(subset='prehistory',
                     extra_transforms=[MultipleChoiceTransform(
                         prompt_prefix="Please answer with the letter of the correct answer.\n")])

Converting dataset...: 100%|██████████| 324/324 [00:00<00:00, 41206.66it/s]


In [6]:
print(dataset[0].question)
from retrieval_qa_benchmark.models.openai import GPT, ChatGPT
model = ChatGPT.build(model_name='model')
model.generate(dataset[0].question, temperature=0)

Please answer with the letter of the correct answer.

Unlike most other early civilizations, Minoan culture shows little evidence of:
A. trade.	B. warfare.	C. the development of a common religion.	D. conspicuous consumption by elites.



'Here is the correct answer: D. conspicuous consumption by elites.'